In [ ]:
!apt-get install -y glpk-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 35 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libglpk40 amd64 5.0-1 [361 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 glpk-ut

In [ ]:
from pyomo.environ import *

nodes = list(range(1, 28))
depots = list(range(1, 6))
clients = list(range(6, 28))
vehicles = list(range(1, 6))

Q_CAPACITY = 4500

coord_x = {
    1:244, 2:263, 3:234, 4:225, 5:320, 6:295, 7:301, 8:309, 9:217, 10:218,
    11:282, 12:242, 13:230, 14:249, 15:256, 16:265, 17:267, 18:259, 19:315, 20:329,
    21:318, 22:329, 23:267, 24:275, 25:303, 26:208, 27:326
}
coord_y = {
    1:248, 2:218, 3:210, 4:246, 5:221, 6:272, 7:258, 8:260, 9:274, 10:278,
    11:267, 12:249, 13:262, 14:268, 15:267, 16:257, 17:242, 18:265, 19:233, 20:252,
    21:252, 22:224, 23:213, 24:192, 25:201, 26:217, 27:181
}
demand = {
     6:125,  7:84,   8:60,   9:500, 10:300,
    11:175, 12:350, 13:150, 14:1100,15:4100,
    16:225, 17:300, 18:250, 19:500, 20:150,
    21:100, 22:250, 23:120, 24:600, 25:500,
    26:175, 27:75
}

distance = {(i, j): ((coord_x[i] - coord_x[j])**2 + (coord_y[i] - coord_y[j])**2)**0.5
            for i in nodes for j in nodes}

model = ConcreteModel()

# Sets
model.NODES = Set(initialize=nodes)
model.DEPOTS = Set(initialize=depots)
model.CLIENTS = Set(initialize=clients)
model.VEHICLES = Set(initialize=vehicles)

# parametros
model.coord_x = Param(model.NODES, initialize=coord_x)
model.coord_y = Param(model.NODES, initialize=coord_y)
model.demand = Param(model.CLIENTS, initialize=demand)
model.Q_CAPACITY = Param(initialize=Q_CAPACITY)
model.c = Param(model.NODES, model.NODES, initialize=distance)

# Variables
model.x = Var(model.NODES, model.NODES, model.VEHICLES, within=Binary)
model.u = Var(model.CLIENTS, model.VEHICLES, within=NonNegativeReals)

# Funcion Objetivo
def total_cost(model):
    return sum(model.c[i, j] * model.x[i, j, k]
               for i in model.NODES for j in model.NODES for k in model.VEHICLES)
model.TotalCost = Objective(rule=total_cost, sense=minimize)

# restricciones

def flow_balance(model, i, k):
    return sum(model.x[j, i, k] for j in model.NODES) == sum(model.x[i, j, k] for j in model.NODES)
model.FlowBalance = Constraint(model.CLIENTS, model.VEHICLES, rule=flow_balance)

def start_depot(model, d, k):
    return sum(model.x[d, j, k] for j in model.CLIENTS) <= 1
model.StartDepot = Constraint(model.DEPOTS, model.VEHICLES, rule=start_depot)

def return_depot(model, d, k):
    return sum(model.x[i, d, k] for i in model.CLIENTS) <= 1
model.ReturnDepot = Constraint(model.DEPOTS, model.VEHICLES, rule=return_depot)

def unique_visit(model, j):
    return sum(model.x[i, j, k] for i in model.NODES for k in model.VEHICLES) == 1
model.UniqueVisit = Constraint(model.CLIENTS, rule=unique_visit)

def subtour_elimination(model, i, j, k):
    if i == j:
        return Constraint.Skip
    return model.u[i, k] - model.u[j, k] + model.demand[j] * model.x[i, j, k] <= model.Q_CAPACITY - model.demand[j]
model.SubtourElimination = Constraint(model.CLIENTS, model.CLIENTS, model.VEHICLES, rule=subtour_elimination)

def load_lower(model, i, k):
    return model.u[i, k] >= model.demand[i] * sum(model.x[j, i, k] for j in model.NODES)
model.LoadLower = Constraint(model.CLIENTS, model.VEHICLES, rule=load_lower)

def load_upper(model, i, k):
    return model.u[i, k] <= model.Q_CAPACITY
model.LoadUpper = Constraint(model.CLIENTS, model.VEHICLES, rule=load_upper)

def no_self_loops(model, i, k):
    return model.x[i, i, k] == 0
model.NoSelfLoops = Constraint(model.NODES, model.VEHICLES, rule=no_self_loops)

def one_start(model, k):
    return sum(model.x[d, j, k] for d in model.DEPOTS for j in model.CLIENTS) <= 1
model.OneStart = Constraint(model.VEHICLES, rule=one_start)

def one_end(model, k):
    return sum(model.x[i, d, k] for i in model.CLIENTS for d in model.DEPOTS) <= 1
model.OneEnd = Constraint(model.VEHICLES, rule=one_end)

solver = SolverFactory('glpk')
results = solver.solve(model, tee=True)

print(f"\nCosto total: {model.TotalCost():.2f}")
for k in model.VEHICLES:
    print(f"\nRuta del vehículo {k}:")
    for i in model.NODES:
        for j in model.NODES:
            if model.x[i, j, k].value == 1:
                print(f"  {i} → {j}")


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmp8dvtlxxs.glpk.raw --wglp /tmp/tmpftj0x4kr.glpk.glp --cpxlp
 /tmp/tmpfdsliiq6.pyomo.lp
Reading problem data from '/tmp/tmpfdsliiq6.pyomo.lp'...
/tmp/tmpfdsliiq6.pyomo.lp:36991: warning: lower bound of variable 'x6445' redefined
/tmp/tmpfdsliiq6.pyomo.lp:36991: warning: upper bound of variable 'x6445' redefined
2857 rows, 3755 columns, 21145 non-zeros
3645 integer variables, all of which are binary
40636 lines were read
Writing problem data to '/tmp/tmpftj0x4kr.glpk.glp'...
33992 lines were written
GLPK Integer Optimizer 5.0
2857 rows, 3755 columns, 21145 non-zeros
3645 integer variables, all of which are binary
Preprocessing...
2612 rows, 3520 columns, 20680 non-zeros
3410 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  4.100e+03  ratio =  4.100e+03
GM: min|aij| =  4.815e-01  max|aij| =  2.077e+00  ratio =  4.313e+00
EQ: min|aij| =  2.340e-01  max|aij|

## Vecino más Cercano

In [ ]:
import math
from collections import defaultdict

# Función de distancia euclidiana
def dist(i, j):
    return math.hypot(coord_x[i] - coord_x[j], coord_y[i] - coord_y[j])

clientes_disponibles = set(clients)
rutas = defaultdict(list)
cargas = defaultdict(int)
costo_total = 0

for v, depot in enumerate(depots, start=1):
    actual = depot
    rutas[v].append(actual)
    carga = 0

    while True:
        vecinos = [c for c in clientes_disponibles if carga + demand[c] <= Q_CAPACITY]
        if not vecinos:
            break

        siguiente = min(vecinos, key=lambda c: dist(actual, c))
        rutas[v].append(siguiente)
        costo_total += dist(actual, siguiente)
        carga += demand[siguiente]
        cargas[v] = carga
        clientes_disponibles.remove(siguiente)
        actual = siguiente

    rutas[v].append(depot)
    costo_total += dist(actual, depot)

# Mostrar rutas y costos
for v in rutas:
    ruta_str = ' → '.join(map(str, rutas[v]))
    print(f"Vehículo {v} (Carga {cargas[v]}): {ruta_str}")

print(f"\nCosto total aproximado con heurística: {costo_total:.2f}")


Vehículo 1 (Carga 4470): 1 → 12 → 13 → 9 → 10 → 14 → 18 → 16 → 17 → 23 → 24 → 25 → 27 → 1
Vehículo 2 (Carga 4484): 2 → 15 → 11 → 6 → 7 → 2
Vehículo 3 (Carga 1235): 3 → 26 → 19 → 22 → 20 → 21 → 8 → 3
Vehículo 4 (Carga 0): 4 → 4
Vehículo 5 (Carga 0): 5 → 5

Costo total aproximado con heurística: 779.91


## Greedy

In [ ]:
from collections import defaultdict
import math

def dist(i, j):
    return math.hypot(coord_x[i] - coord_x[j], coord_y[i] - coord_y[j])

clientes_disponibles = set(clients)
rutas = defaultdict(list)
cargas = defaultdict(int)
costo_total = 0

for v, depot in enumerate(depots, start=1):
    actual = depot
    rutas[v].append(actual)
    carga = 0

    while True:
        # Clientes viables por capacidad
        candidatos = [c for c in clientes_disponibles if carga + demand[c] <= Q_CAPACITY]
        if not candidatos:
            break

        # Criterio greedy: distancia / demanda
        siguiente = min(
            candidatos,
            key=lambda c: dist(actual, c) / demand[c]  # eficiencia costo/demanda
        )

        rutas[v].append(siguiente)
        costo_total += dist(actual, siguiente)
        carga += demand[siguiente]
        cargas[v] = carga
        clientes_disponibles.remove(siguiente)
        actual = siguiente

    rutas[v].append(depot)
    costo_total += dist(actual, depot)

# Mostrar las rutas y el costo
for v in rutas:
    print(f"Vehículo {v} (Carga {cargas[v]}): {' → '.join(map(str, rutas[v]))}")

print(f"\nCosto total estimado con heurística greedy: {costo_total:.2f}")


Vehículo 1 (Carga 4500): 1 → 15 → 18 → 13 → 1
Vehículo 2 (Carga 4494): 2 → 14 → 12 → 9 → 10 → 24 → 25 → 19 → 22 → 20 → 21 → 8 → 7 → 2
Vehículo 3 (Carga 1195): 3 → 17 → 16 → 11 → 6 → 23 → 26 → 27 → 3
Vehículo 4 (Carga 0): 4 → 4
Vehículo 5 (Carga 0): 5 → 5

Costo total estimado con heurística greedy: 923.57
